In [57]:
import os


In [58]:
%pwd

'/Users/vivekrauniyar/Documents/DS Stuffs/Python_DS/Projects/Kidney-Disease-Classification-project'

In [59]:
%pwd

'/Users/vivekrauniyar/Documents/DS Stuffs/Python_DS/Projects/Kidney-Disease-Classification-project'

In [60]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/VivekRauniyar18/Kidney-Disease-Classification-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "VivekRauniyar18"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "3e56bc451f33247f3fc88ff9070d4dd17ffde709"

In [61]:
import tensorflow as tf

In [62]:
from pathlib import Path
Path("artifacts/training/model.h5").exists()


True

In [63]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [64]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    experiment_name: str
    run_name: str
    mlflow_uri: str


In [65]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [66]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(self.config.data_ingestion.unzip_dir),
            mlflow_uri=self.config.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            experiment_name="kidney-disease-experiment",
            run_name="run-1"
        )
        return eval_config


In [67]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [68]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"

        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            shuffle = False,
            subset = "validation",
            
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Model registry does not work with file store

            if tracking_url_type_store !="file":

                mlflow.keras.log_model(self.model, "model",  registered_model_name = "VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
                    


        

In [69]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e


Reading YAML from path: config/config.yaml
[2025-08-10 22:21:44,801: INFO: common: yaml file: config/config.yaml loaded successfully]
Reading YAML from path: params.yaml
[2025-08-10 22:21:44,803: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-10 22:21:44,804: INFO: common: created directory at: artifacts]
Found 4416 images belonging to 2 classes.
136/276 [=============>................] - ETA: 2:17 - loss: 22.7775 - accuracy: 0.0000e+00

2025-08-10 22:23:58.622647: W tensorflow/core/framework/op_kernel.cc:1816] UNKNOWN: UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x34a50d5e0>
Traceback (most recent call last):

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/keras/src/engine/data_adapter.py", line 1064, in generat

138/276 [==============>...............] - ETA: 2:15 - loss: 22.7830 - accuracy: 0.0000e+00

UnknownError: Graph execution error:

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x34a50d5e0>
Traceback (most recent call last):

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/keras/src/engine/data_adapter.py", line 1064, in generator_fn
    yield x[i]

  File "/opt/anaconda3/e